In [47]:
import numpy as np

def arm_end(lengths, angles, x0 = 0, y0 = 0):
    x = x0
    y = y0

    last_alpha = angles[0]

    x += lengths[0] * np.cos(last_alpha)
    y += lengths[0] * np.sin(last_alpha)

    for i in range(1, len(angles)):
        length = lengths[i]
        angle = angles[i]

        alpha = angle - np.pi + last_alpha

        x += length * np.cos(alpha)
        y += length * np.sin(alpha)

    return x,y

def distance(a, b):
    return np.sqrt((a[0]-b[0])**2 + (a[1] - b[1])**2)


def degrees_to_radians(degrees):
    return list(map(lambda x: x/180 * np.pi, degrees))


lengths = [1, 1, 5]
angles = [45, 180, 225]

arm_end(lengths, degrees_to_radians(angles))

IndexError: list index out of range

In [ ]:
def objective_function(angles, lengths, target):
    end = arm_end(lengths, angles)
    return - distance(end, target)

def init_population(population_size, limits):
    def random_individual():
        return [np.random.rand() * (lim[1] - lim[0]) + lim[0] for lim in limits]

    return np.array([random_individual() for _ in range(population_size)])

def mutation(P, S, limits, tau, tau0):
    N = len(P)
    n = len(P[0])

    def mutate_ind(i):
        epsilon0 = np.random.randn() * tau0
        epsilons = np.random.randn(n) * tau
        new_S = S[i] * np.exp(epsilons + epsilon0)
        epsilons = np.random.randn(n) * new_S
        new_P = (P[i] + epsilons) % (2*np.pi)
        return new_P, new_S

    for i in range(N):
        Pi, Si = mutate_ind(i)

        for j in range(len(Pi)):
            if Pi[j] < limits[j][0]:
                Pi[j] = limits[j][0]
            elif Pi[j] > limits[j][1]:
                Pi[j] = limits[j][1]

        P[i], S[i] = Pi, Si



def ES(target, lengths, limits, population_size, number_of_iterations, number_of_offspring, number_of_parents, sigma, tau, tau_0, log_frequency=1):
    chromosome_length = len(limits)

    log_objective_values = np.empty((number_of_iterations, 4))
    log_best_solutions = np.empty((number_of_iterations, chromosome_length))
    log_best_sigmas = np.empty((number_of_iterations, chromosome_length))

    # generating an initial population
    current_population_solutions = init_population(population_size, limits)
    current_population_sigmas = sigma * np.ones((population_size, chromosome_length))

    # evaluating the objective function on the current population
    current_population_objective_values = objective_function(current_population_solutions, lengths, target)

    for t in range(number_of_iterations):

        # selecting the parent indices by the roulette wheel method
        fitness_values = current_population_objective_values - current_population_objective_values.min()
        if fitness_values.sum() > 0:
            fitness_values = fitness_values / fitness_values.sum()
        else:
            fitness_values = 1.0 / population_size * np.ones(population_size)
        parent_indices = np.random.choice(population_size, (number_of_offspring, number_of_parents), True, fitness_values).astype(np.int64)

        # creating the children population by Global Intermediere Recombination
        children_population_solutions = np.zeros((number_of_offspring, chromosome_length))
        children_population_sigmas = np.zeros((number_of_offspring, chromosome_length))
        for i in range(number_of_offspring):
            children_population_solutions[i, :] = current_population_solutions[parent_indices[i, :], :].mean(axis=0)
            children_population_sigmas[i, :] = current_population_sigmas[parent_indices[i, :], :].mean(axis=0)

        # mutating the children population by adding random gaussian noise
        # children_population_sigmas = children_population_sigmas * np.exp(tau * np.random.randn(number_of_offspring, chromosome_length) + tau_0 * np.random.randn(number_of_offspring, 1))
        # children_population_solutions = children_population_solutions + children_population_sigmas * np.random.randn(number_of_offspring, chromosome_length)

        mutation(children_population_solutions, children_population_sigmas, limits, tau, tau_0)

        # evaluating the objective function on the children population
        children_population_objective_values = objective_function(current_population_solutions, lengths, target)

        # replacing the current population by (Mu + Lambda) Replacement
        current_population_objective_values = np.hstack([current_population_objective_values, children_population_objective_values])
        current_population_solutions = np.vstack([current_population_solutions, children_population_solutions])
        current_population_sigmas = np.vstack([current_population_sigmas, children_population_sigmas])

        I = np.argsort(current_population_objective_values)[::-1]
        current_population_solutions = current_population_solutions[I[:population_size], :]
        current_population_sigmas = current_population_sigmas[I[:population_size], :]
        current_population_objective_values = current_population_objective_values[I[:population_size]]

        # recording some statistics
        if best_solution_objective_value < current_population_objective_values[0]:
            best_solution = current_population_solutions[0, :]
            best_solution_objective_value = current_population_objective_values[0]
        log_objective_values[t, :] = [current_population_objective_values.min(), current_population_objective_values.max(), current_population_objective_values.mean(), current_population_objective_values.std()]
        log_best_solutions[t, :] = current_population_solutions[0, :]
        log_best_sigmas[t, :] = current_population_sigmas[0, :]

        if np.mod(t, log_frequency) == 0:
            print("Iteration %04d : best score = %0.8f, mean score = %0.8f." % (t, log_objective_values[:t+1, 1].max(), log_objective_values[t, 2]))

    return best_solution_objective_value, best_solution, log_objective_values, log_best_solutions, log_best_sigmas
